In [9]:
import random
from owlready2 import *
import logging

# Logging setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration
dataset = "OWL2DL-1"  # change to your ontology file without .owl
# dataset = "pizza"  # change to your ontology file without .owl
default_instances_per_class = 1
num_relations_per_subject = 1

# Load ontology
onto = get_ontology(f"{dataset}.owl").load()
logger.info(f"Loaded ontology: {onto.base_iri}")

# Helper: determine if a class is instantiable
def is_instantiable(cls):
    from owlready2 import ThingClass, Thing, Nothing
    if not isinstance(cls, ThingClass):
        return False
    if cls in [Thing, Nothing]:
        return False
    # Skip helper classes like ValuePartition or enumerations
    for base in cls.is_a:
        if isinstance(base, ThingClass) and base.name in ["ValuePartition"]:
            return False
    return True

# Detect instantiable classes
instantiable_classes = [cls for cls in onto.classes() if is_instantiable(cls)]
logger.info(f"Detected {len(instantiable_classes)} instantiable classes.")

# Automatically create class_config
class_config = {cls.name: default_instances_per_class for cls in instantiable_classes}

# Detect object properties and build relation_config
relation_config = []
for prop in onto.object_properties():
    # Get domain and range classes
    domains = [d for d in prop.domain if isinstance(d, ThingClass) and is_instantiable(d)]
    ranges = [r for r in prop.range if isinstance(r, ThingClass) and is_instantiable(r)]
    for d in domains:
        for r in ranges:
            relation_config.append((d.name, prop.name, r.name))
logger.info(f"Detected {len(relation_config)} possible relations.")

# Generate instances
instances = {}

with onto:
    # Create individuals for each class
    for cls_name, n in class_config.items():
        cls = onto.search_one(iri=f"*{cls_name}")  # safer than onto[cls_name]
        if cls is None:
            logger.warning(f"Class '{cls_name}' not found or not instantiable. Skipping.")
            continue

        # Confirm cls is a ThingClass
        from owlready2 import ThingClass
        if not isinstance(cls, ThingClass):
            logger.warning(f"Class '{cls_name}' is not a ThingClass. Skipping.")
            continue

        # Safe instance creation
        individuals = [cls(f"{cls_name}_{i}") for i in range(n)]
        instances[cls_name] = individuals
        logger.info(f"✅ Created {n} instances of {cls_name}")

    # Assign object property relations
    for subj_class, prop_name, obj_class in relation_config:
        if subj_class not in instances or obj_class not in instances:
            continue
        try:
            prop = onto[prop_name]
        except KeyError:
            continue
        if not isinstance(prop, ObjectPropertyClass):
            continue

        subjects = instances[subj_class]
        objects = instances[obj_class]

        for subj in subjects:
            valid_objects = [obj for obj in objects if any(cls in obj.is_instance_of for cls in prop.range)]
            if not valid_objects:
                continue

            num_to_assign = min(num_relations_per_subject, len(valid_objects))
            chosen_objs = random.sample(valid_objects, num_to_assign)

            for obj in chosen_objs:
                # Safely check that the property is valid on this subject
                if getattr(subj, prop_name) is not None:
                    getattr(subj, prop_name).append(obj)
                    logger.info(f"🔗 {subj.name} --{prop_name}--> {obj.name}")
                else:
                    logger.warning(f"⚠️ Cannot assign property '{prop_name}' to '{subj.name}' — skipping.")

# Save ontology with instances
output_file = f"{dataset}.owl"
onto.save(output_file, format="rdfxml")
logger.info(f"Saved ontology with instances to {output_file}")

INFO:__main__:Loaded ontology: https://kracr.iiitd.edu.in/OWL2Bench#
INFO:__main__:Detected 131 instantiable classes.
INFO:__main__:Detected 220 possible relations.
INFO:__main__:✅ Created 1 instances of Person
INFO:__main__:✅ Created 1 instances of Student
INFO:__main__:✅ Created 1 instances of Program
INFO:__main__:✅ Created 1 instances of Department
INFO:__main__:✅ Created 1 instances of University
INFO:__main__:✅ Created 1 instances of AssistantProfessor
INFO:__main__:✅ Created 1 instances of AssociateProfessor
INFO:__main__:✅ Created 1 instances of Publication
INFO:__main__:✅ Created 1 instances of ClericalStaff
INFO:__main__:✅ Created 1 instances of College
INFO:__main__:✅ Created 1 instances of CollegeDiscipline
INFO:__main__:✅ Created 1 instances of Organization
INFO:__main__:✅ Created 1 instances of Faculty
INFO:__main__:✅ Created 1 instances of FullProfessor
INFO:__main__:✅ Created 1 instances of Lecturer
INFO:__main__:✅ Created 1 instances of Engineering
INFO:__main__:✅ Crea